In [1]:
import joblib
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering

Z:\PPMI_Data\Excels\CollaborativeFiltering


In [2]:
# Initialize an empty list to store the tests and scores
tests_scores = []

# Get the number of tests
while True:
    num_tests = int(input('Enter the number of tests (minimum 3, maximum 45): '))
    if num_tests >= 3 and num_tests <= 45:
        break
    print("Invalid input, please enter a number between 3 and 45")

# Get the name and score of each test
for i in range(num_tests):
    name = input(f'Enter the name of test {i + 1}: ')
    score = int(input(f"What is the score for {name}: "))
    tests_scores.append([name, score])

# Initialize an empty list to store the predictions
predictions = []

# Predict for each test
for i in range(num_tests):
    name = tests_scores[i][0]
    user_data = tests_scores[i][1]
    user_data = np.array(user_data).reshape(-1, 1)

    # Load the AdaBoost model from the file
    #adaBoost_model = joblib.load(name + '_AdaBoost_model.joblib')

    # Use the loaded model for predictions or further processing
    #adaBoost_prediction = adaBoost_model.predict(user_data)
    #print(f"For the test {name}, user may belong to: {adaBoost_prediction[0]} using AdaBoost model")

    # Load the Lasso model from the file
    LASSO_model = joblib.load(name + '_Lasso_model.joblib')

    # Use the loaded model for predictions or further processing
    Lasso_prediction = LASSO_model.predict(user_data)
    
    print(f"For the test {name}, user may belong to: {Lasso_prediction[0]} using Lasso model")

    # Load the SVM Linear model from the file
    svmL_model = joblib.load(name + '_svmL_model.joblib')

    # Use the loaded model for predictions or further processing
    SVM_prediction = svmL_model.predict(user_data)
    print(f"For the test {name}, user may belong to: {SVM_prediction[0]} using SVM model")
    
    # Load the Elastic Net model from the file
    enet_model = joblib.load(name + '_enet_model.joblib')

    # Use the loaded model for predictions or further processing
    enet_prediction = enet_model.predict(user_data)
    print(f"For the test {name}, user may belong to: {enet_prediction[0]} using Elastic Net model")
    
    # Load the Bagging model from the file
    #Bag_model = joblib.load(name + '_bag_model.joblib')

    # Use the loaded model for predictions or further processing
    #Bag_prediction = Bag_model.predict(user_data)
    #print(f"For the test {name}, user may belong to: {Bag_prediction[0]} using Bagging model")


    # Add the predictions to the list
    predictions.append([ Lasso_prediction[0], SVM_prediction[0],enet_prediction[0]])

# Convert the list into a dataframe
feat_pred_df = pd.DataFrame(predictions, columns=["Lasso", "SVM Linear", "Elastic Net"])


Enter the number of tests (minimum 3, maximum 45): 3
Enter the name of test 1: Apathy
What is the score for Apathy: 2
Enter the name of test 2: Clock
What is the score for Clock: 3
Enter the name of test 3: UPSIT
What is the score for UPSIT: 39
For the test Apathy, user may belong to: Patient using Lasso model
For the test Apathy, user may belong to: Patient using SVM model
For the test Apathy, user may belong to: Patient using Elastic Net model
For the test Clock, user may belong to: Patient using Lasso model
For the test Clock, user may belong to: Patient using SVM model
For the test Clock, user may belong to: Patient using Elastic Net model
For the test UPSIT, user may belong to: Healthy using Lasso model
For the test UPSIT, user may belong to: Healthy using SVM model
For the test UPSIT, user may belong to: Healthy using Elastic Net model


In [3]:
feat_pred_df.head()

,Lasso,SVM Linear,Elastic Net
0,Patient,Patient,Patient
1,Patient,Patient,Patient
2,Healthy,Healthy,Healthy


In [4]:
counts = feat_pred_df.apply(pd.Series.value_counts, axis=1).fillna(0).astype(int)
# get the index (value) of the maximum occurrence in each row
result = counts.idxmax(axis=1)

# create a new column with the result
max_inAll = pd.DataFrame()
max_inAll["Maximum_Occurrence"] = pd.DataFrame(result)

# view the final dataframe
max_inAll.head()


,Maximum_Occurrence
0,Patient
1,Patient
2,Healthy


In [5]:
max_inAll = max_inAll.iloc[:, ]
counts = max_inAll['Maximum_Occurrence'].value_counts()
most_occurring_value = counts.index[0]
print("The user might fall under category of", most_occurring_value)

The user might fall under category of Patient


In [6]:
# Count the occurrences of each category
patient_count = ((feat_pred_df == "Patient").sum(axis=1) > 0).sum()
healthy_count = ((feat_pred_df == "Healthy").sum(axis=1) > 0).sum()

# Calculate the percentage of occurrence for each category
total_count = len(feat_pred_df)
patient_percent = (patient_count / total_count)
healthy_percent = (healthy_count / total_count)

# Print the results
print("Patient values in IndiMod:", patient_percent)
print("Healthy values in IndiMod:", healthy_percent)

Patient values in IndiMod: 0.6666666666666666
Healthy values in IndiMod: 0.3333333333333333


In [8]:
# Count the occurrences of each category
patient_count = ((feat_pred_df == "Patient").sum(axis=1) > 0).sum()
healthy_count = ((feat_pred_df == "Healthy").sum(axis=1) > 0).sum()

# Calculate the percentage of occurrence for each category
total_count = len(feat_pred_df)
patient_percent = (patient_count / total_count)
healthy_percent = (healthy_count / total_count)

# Determine which category has the highest percentage
if patient_percent > healthy_percent:
    category = "Patient"
    percentage = patient_percent
else:
    category = "Healthy"
    percentage = healthy_percent

# Create a new dataframe with the result
result_df_IndiMod = pd.DataFrame({"Category": [category], "Percentage": [percentage]})

# Print the result dataframe
print(result_df_IndiMod)


  Category  Percentage
0  Patient    0.666667
